In [10]:
import pickle
import numpy as np
from vcdist_funcs import *
import time

# file_path = '/media/zzs/4TB/qingliu/qing_intermediate/feat_pickle/'
file_path = '/mnt/4T-HD/qing/voting_data/feat_pickle_all_k208/'
# file_path = '/mnt/4T-HD/qing/intermediate/feat_pickle/'

magic_thh = 0.7

layer_feature_dist = []
sub_type = []
view_point = []
# objects = ['car', 'aeroplane', 'bicycle', 'bus', 'motorbike', 'train']
objects = ['car']
for oo in objects:
    fname = file_path + 'res_info_' + oo + '_train.pickle'
    print('loading object {0}'.format(oo))
    with open(fname, 'rb') as fh:
        l, s, v = pickle.load(fh)
        layer_feature_dist += l
        sub_type += s
        view_point += v
        
N = len(sub_type)
print('total number of instances {0}'.format(N))

N=100
layer_feature_dist = layer_feature_dist[0:N]
sub_type = sub_type[0:N]
view_point = view_point[0:N]


loading object car
total number of instances 1000


In [11]:
layer_feature_dist[0].shape

(208, 21, 14)

In [22]:
layer_feature_b = [None for nn in range(N)]
patch_feature_dist = []
patch_feature_b = []
pos_rec = []
patch_size = [7,7]  # h, w of patch
for nn in range(N):
    layer_feature_b[nn] = (layer_feature_dist[nn]<magic_thh).astype(int)
    w,h = layer_feature_b[nn].shape[1:]
    
    for ww in range(0, w-patch_size[1]+1, 2):
        for hh in range(0, h-patch_size[0]+1, 2):
            patch_feature_dist.append(layer_feature_dist[nn][:,ww:ww+patch_size[1], hh:hh+patch_size[0]])
            patch_feature_b.append(layer_feature_b[nn][:,ww:ww+patch_size[1], hh:hh+patch_size[0]])
            pos_rec.append([nn, ww, hh])
            
            
print(len(patch_feature_dist))
            

4648


In [24]:
patch_feature_dist_f = np.array([dd.reshape(-1,) for dd in patch_feature_dist])
patch_feature_dist_f.shape

(4648, 10192)

In [25]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=50, init='k-means++', random_state=99)
assignment = km.fit_predict(patch_feature_dist_f)
centers = km.cluster_centers_
print(centers.shape)

KeyboardInterrupt: 